In [3]:
import pandas as pd
import numpy as np

## User Data

In [4]:
#def user_graph():
users  = pd.read_csv('./clean/accounts.csv', on_bad_lines='skip', low_memory=False, lineterminator='\n') #We are encoutering some bad lines creating some error so we skip over them
cat=["Id","Gender", "Social network"]
num=['Country id','City id','Followers',"Engagement","Posts","Estimated reach","Avg. views per video", "Avg. comments per post", "Avg. likes per post", "Avg. posts per week"]
n = users.shape[0]
clean_df = users[cat+num][:n]
clean_df = clean_df.rename(columns={'City id': 'City','Country id':'Country'})
for i in clean_df.columns:
    clean_df = clean_df.rename(columns={i: i.replace(" ","_")})
#clean_df = clean_df.set_index("Id")

In [13]:
#City-ID mapping
city_ids = pd.read_csv('../SocialTalk 2023/cities.csv', low_memory=False) 
city_ids['Name'] = city_ids['Name'].str.replace(" ","_")#Replace white space by "_"

clean_df['City'] = clean_df['City'].map(city_ids.set_index('Id')['Name'].to_dict())

In [14]:
#Country-ID mapping
country_ids = pd.read_csv('../SocialTalk 2023/countries.csv', low_memory=False)
country_ids['Name'] = country_ids['Name'].str.replace(" ","_")#Replace white space by "_"

clean_df['Country'] = clean_df['Country'].map(country_ids.set_index('Id')['Name'].to_dict())

In [15]:
# Removing outlier
users.loc[users['Posts'] == 0,'Posts'] = np.nan 
users.loc[users['Avg. views per video'] == 0,'Avg. views per video'] = np.nan
users.loc[users['Estimated reach'] == 0,'Estimated reach'] = np.nan
users.loc[users["Avg. comments per post"] == 0 ,"Avg. comments per post"] = np.nan
users.loc[users["Avg. likes per post"] == 0 ,"Avg. likes per post"] = np.nan

q_low = users["Avg. posts per week"].quantile(0.01)
q_hi  = users["Avg. posts per week"].quantile(0.99)
users.loc[(users["Avg. posts per week"] < q_hi) & (users["Avg. posts per week"] > q_low) ,"Avg. posts per week"] = np.nan

## Audience Data

In [16]:
audience_genders = pd.read_csv('./clean/accounts-audience-genders.csv')[:n]
for i in audience_genders.columns[1:]:
    audience_genders.rename(columns = {i:'Audience-Gender-' + i.replace(' ','_')}, inplace = True)

In [17]:
audience_ages = pd.read_csv('./clean/accounts-audience-ages.csv')
audience_ages = audience_ages[:n]
for i in audience_ages.columns:
    if(i != 'Account Id'):
        audience_ages.rename(columns = {i:'Audience-Age-' + i.replace(' ','_')}, inplace = True)

In [18]:
def get_real_audience_data(df,audience_data):

    #merging audience gender distribution and accounts data
    print("Merging")
    df = df.merge(audience_data,how='left' ,left_on='Id',right_on = 'Account Id')
    df.drop(['Account Id'], axis=1,inplace=True)
    
    for audience_data_column in audience_data.columns:
        if(audience_data_column != 'Account Id'):
            print('Turning '+ audience_data_column + ' into raw numbers')
            df[audience_data_column] = df[audience_data_column] * df['Followers'] # Turning into raw numbers
            print('Binning')
            df[audience_data_column] = pd.qcut(df[audience_data_column], q=[0, .25, .75, 1], labels=['Low', 'Medium', 'High']) #binning 
    return df

In [19]:
clean_df = get_real_audience_data(clean_df,audience_genders)

Merging
Turning Audience-Gender-MALE_% into raw numbers
Binning
Turning Audience-Gender-FEMALE_% into raw numbers
Binning


In [20]:
clean_df = get_real_audience_data(clean_df,audience_ages)

Merging
Turning Audience-Age-13-17_% into raw numbers
Binning
Turning Audience-Age-18-24_% into raw numbers
Binning
Turning Audience-Age-25-34_% into raw numbers
Binning
Turning Audience-Age-35-44_% into raw numbers
Binning
Turning Audience-Age-45-64_% into raw numbers
Binning
Turning Audience-Age-65-_% into raw numbers
Binning


ValueError: Bin edges must be unique: array([0.00000000e+00, 0.00000000e+00, 1.14154008e+01, 4.41355904e+05]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [21]:
clean_df[]

,Id,Gender,Social_network,Country,City,Followers,Engagement,Posts,Estimated_reach,Avg._views_per_video,Avg._comments_per_post,Avg._likes_per_post,Avg._posts_per_week,Audience-Gender-MALE_%,Audience-Gender-FEMALE_%
0,151,Male,Instagram,Spain,Lisbon,154.0,0,3,153,0.0,0.0,0.0,1,Low,Low
1,152,Male,Instagram,United_States,Los_Angeles,22625181.0,39322588,715,1062314,1062313.0,2100.0,434816.0,9,High,High
2,153,Male,Instagram,Saudi_Arabia,Manchester,550332150.0,1032630249,3443,18275483,18275482.0,119545.0,9809591.0,4,High,High
3,154,Male,Instagram,Brazil,São_Paulo,71552085.0,59144108,2953,1435572,1435571.0,2731.0,588709.0,4,High,High
4,155,Female,Instagram,United_States,NaN,39861444.0,17922824,295,3587530,0.0,1767.0,906753.0,2,High,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931193,2936836,NaN,Instagram,Ukraine,Zaporizhzhia,9985.0,0,0,0,0.0,0.0,0.0,0,NaN,NaN
2931194,2936837,NaN,Instagram,Malaysia,NaN,3404.0,0,0,0,0.0,0.0,0.0,0,NaN,NaN
2931195,2936838,NaN,Instagram,Argentina,Mendoza,35093.0,0,0,0,0.0,0.0,0.0,0,NaN,NaN
2931196,2936839,Female,Instagram,Sri_Lanka,NaN,9460.0,0,0,0,0.0,0.0,0.0,0,NaN,NaN


In [152]:
# Binning Numeric columns
numerics = [ 'int64', 'float64']
num_columns=clean_df.select_dtypes(include=numerics).columns[1:]

#numerical edges
for i in num_columns:
    print(i)
    clean_df[i] = pd.qcut(clean_df[i],q=[0, .25, .75, 1], labels=['Low', 'Medium', 'High'],duplicates='drop')

Followers
Engagement
Posts
Estimated_reach
Avg._views_per_video
Avg._comments_per_post
Avg._likes_per_post
Avg._posts_per_week


In [153]:
clean_df['Id'] = clean_df['Id'].apply(lambda x: f'Account:{x}')
clean_df = clean_df.set_index('Id')

## Interests and Brands

In [154]:
brand_ids = pd.read_csv('../SocialTalk 2023/brands.csv', low_memory=False) # Shape (3792, 2)
account_brands = pd.read_csv('../SocialTalk 2023/accounts-brands.csv', low_memory=False)
interest_ids = pd.read_csv('../SocialTalk 2023/interests.csv', low_memory=False) # Shape (60, 2)
account_interests = pd.read_csv('../SocialTalk 2023/accounts-interests.csv', low_memory=False)

In [155]:
brand_ids['Name'] = brand_ids['Name'].str.replace(" ","_")
interest_ids['Name'] = interest_ids['Name'].str.replace(" ","_")

In [156]:
brand_ids['label'] = 'Brand:' + brand_ids['Name'].astype(str)
account_brands['Account Id'] = 'Account:' + account_brands['Account Id'].astype(str)
account_brands['Brand Id'] = account_brands['Brand Id'].map(brand_ids.set_index('Id')['label'].to_dict())
interest_ids['label'] = 'Interest:' + interest_ids['Name'].astype(str)
account_interests['Account Id'] = 'Account:' + account_interests['Account Id'].astype(str)
account_interests['Interest Id'] = account_interests['Interest Id'].map(interest_ids.set_index('Id')['label'].to_dict())

In [157]:
account_brands = account_brands[:n]
account_interests = account_interests[:n]

## Audience fully enriched data (less records)

In [158]:
audience_brands = pd.read_csv('../SocialTalk 2023/accounts-audience-brands.csv')
audience_cities = pd.read_csv('../SocialTalk 2023/accounts-audience-cities.csv')
audience_countries = pd.read_csv('../SocialTalk 2023/accounts-audience-countries.csv')
audience_interests = pd.read_csv('../SocialTalk 2023/accounts-audience-interests.csv')

In [159]:
audience_brands['Account Id'] = 'Account:' + audience_brands['Account Id'].astype(str)
audience_cities['Account Id'] = 'Account:' + audience_cities['Account Id'].astype(str)
audience_countries['Account Id'] = 'Account:' + audience_countries['Account Id'].astype(str)
audience_interests['Account Id'] = 'Account:' + audience_interests['Account Id'].astype(str)

In [160]:
# Down sampling audience_countries and audience_interests
sample_accounts = clean_df.index
audience_countries = audience_countries[audience_countries['Account Id'].isin(sample_accounts)]
audience_interests = audience_interests[audience_interests['Account Id'].isin(sample_accounts)]
audience_brands = audience_brands[audience_brands['Account Id'].isin(sample_accounts)]
audience_cities = audience_cities[audience_cities['Account Id'].isin(sample_accounts)]

In [161]:
# Mapping name and Ids
audience_brands['Brand Id'] = audience_brands['Brand Id'].map(brand_ids.set_index('Id')['Name'].to_dict())
audience_cities['City Id'] = audience_cities['City Id'].map(city_ids.set_index('Id')['Name'].to_dict())
audience_countries['Country Id'] = audience_countries['Country Id'].map(country_ids.set_index('Id')['Name'].to_dict())
audience_interests['Interest Id'] = audience_interests['Interest Id'].map(interest_ids.set_index('Id')['Name'].to_dict())

In [162]:
#Binning on quartile
audience_brands['%'] = pd.qcut(audience_brands['%'], q=[0, .25, .75, 1], labels=['Low', 'Medium', 'High'])
audience_cities['%'] = pd.qcut(audience_cities['%'], q=[0, .25, .75, 1], labels=['Low', 'Medium', 'High'])
audience_countries['%'] = pd.qcut(audience_countries['%'], q=[0, .25, .75, 1], labels=['Low', 'Medium', 'High'])
audience_interests['%'] = pd.qcut(audience_interests['%'], q=[0, .25, .75, 1], labels=['Low', 'Medium', 'High'])

In [163]:
# Creating label

audience_brands['label'] = 'Audience_Brand_'+audience_brands['Brand Id'].astype(str) + ':' + audience_brands['%'].astype(str)
audience_cities['label'] = 'Audience_City_'+audience_cities['City Id'].astype(str) + ':' + audience_cities['%'].astype(str)
audience_countries['label'] = 'Audience_Country_'+audience_countries['Country Id'].astype(str) + ':' + audience_countries['%'].astype(str)
audience_interests['label'] = 'Audience_Interest_'+audience_interests['Interest Id'].astype(str) + ':' + audience_interests['%'].astype(str)


## Edge list

In [164]:
'''
Lets make edge pairs in the given format
(account, f'{column}:{value}') for each column with non-nan values

'''
edges = []
for column in clean_df.columns:
    print(column)
    series_tuples = [
        (account, f'{column}:{value}') for account, value in clean_df[clean_df[column].notna()][column].items()
        ]
    edges.extend(series_tuples)

Gender
Social_network
Country
City
Followers
Engagement
Posts
Estimated_reach
Avg._views_per_video
Avg._comments_per_post
Avg._likes_per_post
Avg._posts_per_week
Audience-Gender-MALE_%
Audience-Gender-FEMALE_%
Audience-Age-13-17_%
Audience-Age-18-24_%
Audience-Age-25-34_%
Audience-Age-35-44_%
Audience-Age-45-64_%
Audience-Age-65-_%


In [165]:
edges += [i for i in account_brands.set_index('Account Id')['Brand Id'].items()]
edges += [i for i in account_interests.set_index('Account Id')['Interest Id'].items()]

In [166]:
#country-city
location_hierarchy = [
    (f'Country:{country}', f'City:{city}') for country, city in clean_df[['Country', 'City']].dropna().drop_duplicates(subset=['Country', 'City'], keep='last').values
]
edges.extend(location_hierarchy)

In [167]:
edges += [i for i in audience_brands.set_index('Account Id')['label'].items()]
edges += [i for i in audience_cities.set_index('Account Id')['label'].items()]
edges += [i for i in audience_countries.set_index('Account Id')['label'].items()]
edges += [i for i in audience_interests.set_index('Account Id')['label'].items()]

In [168]:
temp = pd.DataFrame(edges, columns=['source', 'target'])

In [169]:
temp.to_csv(f'edge_list_{len(temp)}.csv',index=False)

In [170]:
temp['source'].value_counts()

Account:1043           327
Account:3007           312
Account:418            289
Account:1357           283
Account:335            269
                      ... 
Country:Switzerland      1
Country:Australia        1
Country:Canada           1
Country:Argentina        1
Country:Colombia         1
Name: source, Length: 20019, dtype: int64

In [171]:
temp['target'].value_counts()

Social_network:Instagram               17008
Followers:Medium                       10000
Engagement:Medium                      10000
Estimated_reach:Medium                  9998
Avg._likes_per_post:Medium              9998
                                       ...  
Audience_Brand_Cadbury:High                1
Audience_Brand_ALDI:High                   1
Audience_Brand_Next:Low                    1
Audience_Brand_T.G.I._Friday's:Low         1
Audience_Interest_Luxury_Goods:High        1
Name: target, Length: 4047, dtype: int64

In [172]:
# Total nodes
temp['source'].value_counts().shape[0] + temp['target'].value_counts().shape[0]

24066

In [173]:
temp.shape

(601138, 2)

## Inference

In [174]:
from gensim.models import KeyedVectors
import json

In [220]:
with open('user_lookup.json', 'r') as f:
    user_lookup = json.load(f)

In [232]:
model = KeyedVectors.load_word2vec_format('GGvec_model_v1.bin')

In [241]:
positive_list = ['City:London',"Account:152"]
selections_dict = {}
selections_dict['positive'] = positive_list

In [248]:
#fetches data from accounts.csv using positive list
def accounts_data(positive_list=None):
    ac_interests = pd.read_csv("./SocialTalk2023Clean/Clean/accounts-interests.csv")
    ac_brands = pd.read_csv("./SocialTalk2023Clean/Clean/accounts-brands.csv")
    acc = pd.read_csv("./SocialTalk2023Clean/clean/accounts.csv")
    acc_ids=[]
    for i in positive_list:
        node_type, value = i.split(":")
        if node_type == "Account":
            acc_ids.append(int(value))
        elif node_type == "Interest":
            acc_ids.extend(list(ac_interests[ac_interests["Interest"] == value]["Account Id"].values))
        elif node_type == "Brand":
            acc_ids.extend(list(ac_brands[ac_brands["Brand"]==value]["Account Id"].values))
    return acc[acc["Id"].isin(acc_ids)].drop(["label"],axis=1)

In [249]:
def get_recommendations(selections_dict, model, user_lookup, display_n=25):
    results = []
    for node_id, value in model.most_similar_cosmul(**selections_dict, topn=1000000):
        node_parts = node_id.split(':')
        if len(node_parts) != 2:
            node_type = node_parts[0]
            node_key = ' '.join(node_parts[1:])
        else:
            node_type, node_key = node_parts

        if node_type == 'Account':
            try:
                results.append((node_type, user_lookup[node_key]['Name'], value))
                
            except:
                print(node_key)
        elif node_key == 'True':
            pass
        else:
            results.append((node_type, node_id, value))
            
    df = pd.DataFrame(results, columns=['EntityType', 'Entity', 'Score'])
    users_df = accounts_data(selections_dict["positive"])
    return users_df

In [250]:
p = get_recommendations(selections_dict, model, user_lookup, display_n=25)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: DtypeWarning: Columns (0,4,6,19,20) have mixed types.Specify dtype option on import or set low_memory=False.


In [251]:
p

,Id,Social network,Username,Name,Country id,City id,Birthdate,Gender,Followers,Following,...,Estimated reach,Estimated impressions,Avg. posts per week,Avg. posts per month,Avg. likes per post,Avg. engagement per post,Avg. comments per post,Avg. views per video,Relevant tags,Audience credibility
1,152,Instagram,jasonderulo,Jason Derulo,401.0,39.0,NaN,Male,22625181.0,10.0,...,1062314.0,1274777.0,9.0,33.0,434816.0,436917.0,2100.0,1062313.0,"slidin,takethestage,mooski,dancin,gaspedal,mov...",0.6746


In [18]:
users[users.Id == 8637]

,Id,Social network,Username,Name,Country id,City id,Birthdate,Gender,Followers,Following,...,Estimated impressions,Avg. posts per week,Avg. posts per month,Avg. likes per post,Avg. engagement per post,Avg. comments per post,Avg. views per video,Relevant tags,Audience credibility,label
8401,8637,Instagram,wordtomother,wordtomother,400.0,46.0,NaN,NaN,22184.0,0.0,...,12080,1,1,527.0,536.0,9.0,0.0,0,NaN,Account:8637


In [15]:
users[users.Name == 'Marc 🍭💀']

,Id,Social network,Username,Name,Country id,City id,Birthdate,Gender,Followers,Following,...,Estimated impressions,Avg. posts per week,Avg. posts per month,Avg. likes per post,Avg. engagement per post,Avg. comments per post,Avg. views per video,Relevant tags,Audience credibility,label
11341,11577,Instagram,marcfc12,Marc 🍭💀,400.0,NaN,NaN,Male,10517.0,0.0,...,7020,1,1,723.0,696.0,35.0,0.0,0,NaN,Account:11577
